# Solute transport and variable-density modelling
Hi and welcome to workshop six. Fitting all the possible things you can do with solute transport in MF6 in a single session is impossible so we will cover the most commonly applied packages and how to set them up. There is nothing new with regards to how to configure the input data for each package so we will also reference the mf6io.pdf continuously through out the session. This should aid your understanding by filling in the gaps with what won't be demonstrated in here. We will start with the same workshop setup we have been using in each workshop thus far. Then we will construct the Henry Problem, which is a benchmark variable-density problem in groundwater modelling. This will serve as our demonstration model for the complete workshop.


In [ ]:
import os
import sys
import shutil
import platform
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import flopy

print(f"Pandas version = {pd.__version__}")
print(f"Numpy version = {np.__version__}")
print(f"Flopy version = {flopy.__version__}")
print(f"Matplotlib version = {matplotlib.__version__}")

In [ ]:
ws6 = os.path.join('workshop_6') # here we are making a path not creating the folder
gis_f = os.path.join(ws6,'GIS') # creating a sub-directory path for our gis input/output
model_f = os.path.join(ws6,'model') # creating a sub-directory path for our model input/output
plots_f = os.path.join(ws6,'plots') # creating a sub-directory path for our plots
for path in [ws6,gis_f,model_f,plots_f]:
    if os.path.exists(path): # here we are asking if the path exists on the computer. 
        shutil.rmtree(path)# if it does exist, delete it and all the files in it
        os.mkdir(path) # then remake it
    else:
        os.mkdir(path) # if it doesn't exist then make the folder

# The Henry Problem

In [ ]:
# setup a simulation
simulation_name = 'MySim' # setting a name for our simulation
sim = flopy.mf6.MFSimulation(sim_name=simulation_name, sim_ws=model_f, exe_name='mf6') # for a no frills mfsim.nam file

# setup a tdis
pdata = [(0.5,500,1.0)] # one stress period with 500 time steps
tdis = flopy.mf6.ModflowTdis(sim,
                             time_units="seconds",
                             nper=len(pdata),
                             perioddata=pdata) # for the MySim.tdis file, recall this sets a single steady-state stress period

In [ ]:
# setup a flow model
model_name = 'flow' # setting a name for our model
gwf = flopy.mf6.ModflowGwf(sim, modelname=model_name, save_flows=True) # for the flow.nam file

# Multiple IMS instances
The use of a flow-transport combined model requires multiple IMS instances still assigned to the same simulation but registered to different models. The first IMS instance will be automatically registered to the flow model. When we create the transport model will need another but registered to the transport model specifically What is used in the following code block is very similar to the "SIMPLE" configuration of solver settings..


In [ ]:
ims = flopy.mf6.ModflowIms(sim,
                           filename=f"{model_name}.ims", # note specifying filename to align with model not default which aligns with simulation
                           print_option="ALL",
                           outer_dvclose=1e-10,
                           outer_maximum=100,
                           inner_maximum=300,
                           inner_dvclose=1e-10,
                           rcloserecord=1e-6,
                           linear_acceleration="BICGSTAB", # note the other option here is the "CG" for conjugate gradient
                           relaxation_factor=0.97) # used by LU preconditioner - can help convergence # for the MySim.ims file, recall this sets up IMS as 'SIMPLE'

In [ ]:

# A cross sectional model so only one row
nlay = 40  # Number of layers
nrow = 1  # Number of rows
ncol = 80  # Number of columns
system_length = 2.0  # Length of system ($m$)
delr = 0.025  # Column width ($m$)
delc = 1.0  # Row width ($m$)
delv = 0.025  # Layer thickness
top = 1.0  # Top of the model ($m$)
botm = [top - k * delv for k in range(1, nlay + 1)] # list comprehension to get bottom elevations ($m$)

dis = flopy.mf6.ModflowGwfdis(gwf,
                        length_units="cm",
                        nlay=nlay,
                        nrow=nrow,
                        ncol=ncol,
                        delr=delr,
                        delc=delc,
                        top=top,
                        botm=botm)

npf = flopy.mf6.ModflowGwfnpf(gwf,
                        save_specific_discharge=True, # saves x,y,z components of flow so that you can plot flow vectors
                        icelltype=0, # confined system
                        k=864.0) # Hydraulic conductivity ($m d^{-1}$)

ic = flopy.mf6.ModflowGwfic(gwf, strt=1.0) # the actual example has this as the concentartion

oc = flopy.mf6.ModflowGwfoc(gwf,
                       head_filerecord=f"{model_name}.hds",
                       budget_filerecord=f"{model_name}.cbb",
                       saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")])

# get a modelgrid object and plot the grid
mg = gwf.modelgrid
fig = plt.figure(figsize=(7,3)) # we are creating a figure object here so that we can dictate size
ax = fig.add_subplot(1, 1, 1, aspect="equal") # making sure our proportions are kept intact
pxs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0})
pxs.plot_grid()
ax.set_title('DIS Model Grid')
ax.set_xlabel('Length')
ax.set_ylabel('Height')
plt.tight_layout()
# lets save it to our plots folder
figname = os.path.join(plots_f,'DIS_model_grid.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'DIS_model_grid.pdf')
fig.savefig(figname,dpi=300) 

In [ ]:
botm

# Setting the ocean boundary
This is where things will start to change a bit from what we did previously with flow only models. The setup of these boundary conditions represents one of the ways you can setup concentrations in MF6, which is via the auxiliary variables for the stress packages. Here we will use a general head boundary to represent a hydrostatic ocean with a salinity of 35.0 mg/l or 35 kg / 1000 L


In [ ]:
ghbcond = 864.0 * delv * delc / (0.5 * delr) # KA/(L*0.5) # why not KA/L
ghbspd = [[(k, 0, ncol - 1), top, ghbcond, 35.0] for k in range(nlay)] # 35.0 is an auxiliary variable - care to guess why we use 35.0
# so wich side of the model is the ocean?
ghb = flopy.mf6.ModflowGwfghb(gwf,
                        stress_period_data=ghbspd,
                        pname="GHB",               # note the package names are needed later for the transport model linkage
                        auxiliary="CONCENTRATION") # note the use of an auxiliary variable for the SSM package - the name can be anything but we will need to use it later

# Setting discharge to the ocean
The flux from the land to the ocean will be fresh and accordingly be assigned a concentration of zero.

In [ ]:
inflow =5.7024
# inflow = 2.851

welspd = [[(k, 0, 0), inflow / nlay, 0.0] for k in range(nlay)]
wel = flopy.mf6.ModflowGwfwel(gwf,
                        stress_period_data=welspd,
                              pname="WEL",         # note the package names are needed later for the transport model linkage
                        auxiliary="CONCENTRATION") # note we use the same name here indicating that the auxiliary variable referes to the same species
# what do you think would happen if we used a different variable name? Will flopy still build the package? Will the model still run?

# Take a look

In [ ]:
# get a modelgrid object and plot the grid
mg = gwf.modelgrid
fig = plt.figure(figsize=(7,3)) # we are creating a figure object here so that we can dictate size
ax = fig.add_subplot(1, 1, 1, aspect="equal") # making sure our proportions are kept intact
pxs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0})
pxs.plot_grid()
pxs.plot_bc(package=wel, color='blue')
pxs.plot_bc(package=ghb, color='red')
ax.set_title('Model Bounadries')
ax.set_xlabel('Length')
ax.set_ylabel('Height')
plt.tight_layout()
# lets save it to our plots folder
figname = os.path.join(plots_f,'Model_flow_bounds.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'Model_flow_bounds.pdf')
fig.savefig(figname,dpi=300) 

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()
# have we simulated any transport?

# Checking the flow only simulation

In [ ]:
# get the heads 
# load the heads file
headfile = os.path.join(model_f,"flow.hds")
hds = flopy.utils.binaryfile.HeadFile(headfile)

last_step = hds.get_kstpkper()[-1] # how 

h = hds.get_data(last_step)
# get the heads from the final step of the final stress period
# get our contour levels using linspace
levels = np.linspace(np.min(h),np.max(h),20) # we are setting 20 contours across the compete range of heads in the hds array forthe final step

# same as before
mg = gwf.modelgrid
fig = plt.figure(figsize=(7,3)) # we are creating a figure object here so that we can dictate size
ax = fig.add_subplot(1, 1, 1, aspect="equal") # making sure our proportions are kept intact
pxs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0})
fa = pxs.plot_array(h) 
colorbar = plt.colorbar(fa, aspect=30, shrink= 0.8) # create a colorbar
fc = pxs.contour_array(h,
                       levels=levels,
                       colors="w",
                       linewidths=1.0, 
                       linestyles="-")
plt.clabel(fc, fmt="%4.3f", fontsize=7)
ax.set_title('Heads flow only')
ax.set_xlabel('Length')
ax.set_ylabel('Height')
plt.tight_layout()
# lets save it to our plots folder
figname = os.path.join(plots_f,'Henry_heads_flow_only.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'Henry_heads_flow_only.pdf')
fig.savefig(figname,dpi=300) 

# Starting to construct the transport model
One thing that will become obvious is that the flow and transport models share a lot of the same setup, with transport variants of the same packages. This includes the model object, ims object, and the tdis object.


In [ ]:
# virtually identical to the flow model object and includes the same option settings 
# EXCEPT for the object name and class (ModflowGwt)
model_name2 = "trans"
gwt = flopy.mf6.ModflowGwt(sim, modelname=model_name2) # note we pass in the same sim object

In [ ]:
# what does sime look like now
sim

In [ ]:
# Same for the IMS no change needed here 
# EXCEPT for the object name, note class the same here
imsgwt = flopy.mf6.ModflowIms(sim,
                           filename=f"{model_name2}.ims",
                           print_option="ALL",
                           outer_dvclose=1e-10,
                           outer_maximum=100,
                           inner_maximum=300,
                           inner_dvclose=1e-10,
                           rcloserecord=1e-6,
                           linear_acceleration="BICGSTAB", # note the other option here is the "CG" for conjugate gradient
                           relaxation_factor=0.97) # used by LU preconditioner - can help convergence # for the MySim.ims file, recall this sets up IMS as 'SIMPLE'

# register register ims package package to the transport model
sim.register_ims_package(imsgwt, [gwt.name])

In [ ]:
# virtually identical to the flow model object and includes the same option settings 
# EXCEPT for the object name and class (ModflowGwtdis)
# note also that we pass in the # pass in the transport model
tdist = flopy.mf6.ModflowGwtdis(
        gwt,                        
        length_units="cm",
        nlay=nlay,
        nrow=nrow,
        ncol=ncol,
        delr=delr,
        delc=delc,
        top=top,
        botm=botm)

# Mobile storage and mass transfer (porosity, sorption and decay)
All transport models require porosity information so you will always have the is package in your transport model.

In [ ]:
# depending on the model this can have way more settings but they are all either string, boolean or float(array).
# Check the mf6io.pdf document for the format of this pacakge - only one of these packages can be specified per transport model - but how many transport models can we have? 
por = flopy.mf6.ModflowGwtmst(gwt, 
                              porosity=0.35) #  we could also have passed in an array

# Initial concentrations

In [ ]:
# inital concentrations
ict = flopy.mf6.ModflowGwtic(gwt, 
                             strt=35.0) # means we start with an aquifer full of seawater -  we could also have passed in an array

# Advection

In [ ]:
# the mumerical scheme used to calculate advective transport
# check what the other options are.
#We will discuss this package in more detail using the MF6io.pdf document.
adv = flopy.mf6.ModflowGwtadv(gwt, 
                              scheme="UPSTREAM")

# Dispersion and diffusion
We will discuss this package in more detail using the MF6io.pdf document.

In [ ]:
# Note the Henry problem does not use dispersion, which would add lots more input
# Check the mf6io.pdf document for the format of this pacakge
dsp = flopy.mf6.ModflowGwtdsp(gwt, 
                              diffc=0.57024) # value specific to the Henry Problem

# Sink Source Mixing 
This is where the link between the flow models auxiliary variables and the transport model happens. Note this is specific to this type of bounday condition wherein a concentration is assigned to a flux entering the model and the the model will calculate the concentrations exiting he model. This is sometimes referred to as a "third type" soulte boundary. Does it account for dispersive and diffusive flux out of the model? To establish the link with the auxiliary variables in the flow model you have to provide the package and auxilairy variable information. 

In [ ]:
# We start by creating a list of sources defined by string tuples
sourcerecarray = [
        ("GHB", "AUX", "CONCENTRATION"), # package name, source type, variable name in the package
        ("WEL", "AUX", "CONCENTRATION"), # What other source types are there? Why do they exist? Read the mf6io.pdf document
    ]
# now w e build the package and pass in the sources
ssm =  flopy.mf6.ModflowGwtssm(gwt, 
                               sources=sourcerecarray)
# what happens if I also had recharge in the model but didn't assign it an auxliary variable? Will that cause an error? 
# any source or sink in the flow modle that isn't listed is assumed to have zero solute concentration flux in 
# and model calculated concentration flux out

# Output control

In [ ]:
# Same as for flow except the options are specific to soulte transport and the class is different
oct = flopy.mf6.ModflowGwtoc(gwt,
                       budget_filerecord=f"{model_name2}.cbc",
                       concentration_filerecord=f"{model_name2}.ucn",
                       concentrationprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
                       saverecord=[("CONCENTRATION", "ALL")],
                       printrecord=[("CONCENTRATION", "LAST"), ("BUDGET", "LAST")])

# Density
More than one species can contribute to the density calculation. So the inforamtion needed by the BUY package includes the transport model name and the name of the variable representing a solute concentration.

In [ ]:
pd = [(0, 0.7, 0.0, "trans", "concentration")] 
# species number, slope of desnity-concentration line, concentration of the reference density, transport model name, variable name
buy = flopy.mf6.ModflowGwfbuy(gwf, 
                        packagedata=pd)

# Exchange file that links flow and transport

In [ ]:
exch = flopy.mf6.ModflowGwfgwt(sim,                  # pass in the sim
                               exgtype="GWF6-GWT6",  # in this case it is GWF-GWT but we can also have GWF-GWF if linking flow models
                               exgmnamea=gwf.name,   # the flow model name 
                               exgmnameb=gwt.name)   # the transport model name note the order.

In [ ]:
sim.write_simulation()
sim.run_simulation()

In [ ]:
# get the heads 
# load the heads file
headfile = os.path.join(model_f,"flow.hds")
hds = flopy.utils.binaryfile.HeadFile(headfile)

last_step = hds.get_kstpkper()[-1] # how 

h = hds.get_data(last_step)
# get the heads from the final step of the final stress period
# get our contour levels using linspace
levels = np.linspace(np.min(h),np.max(h),20) # we are setting 20 contours across the compete range of heads in the hds array forthe final step

# same as before
mg = gwf.modelgrid
fig = plt.figure(figsize=(7,3)) # we are creating a figure object here so that we can dictate size
ax = fig.add_subplot(1, 1, 1, aspect="equal") # making sure our proportions are kept intact
pxs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0})
fa = pxs.plot_array(h) 
colorbar = plt.colorbar(fa, aspect=30, shrink= 0.8) # create a colorbar
fc = pxs.contour_array(h,
                       levels=levels,
                       colors="w",
                       linewidths=1.0, 
                       linestyles="-")
plt.clabel(fc, fmt="%4.3f", fontsize=7)
ax.set_title('Heads density flow')
ax.set_xlabel('Length')
ax.set_ylabel('Height')
plt.tight_layout()
# lets save it to our plots folder
figname = os.path.join(plots_f,'Henry_heads_density_flow.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'Henry_heads_density_flow.pdf')
fig.savefig(figname,dpi=300) 

In [ ]:
# get the concentrations 
# load the concentration file
concfile = os.path.join(model_f,"trans.ucn")
conc = flopy.utils.binaryfile.HeadFile(concfile, text='conc') 
# Note we are still using a HeadsFile object but reading a concentration file, you must tell it to look for "conc" as the array of interest

last_step = conc.get_kstpkper()[-1] # Now change this to conc instead of hds

c = conc.get_data(last_step)
# get the concentration from the final step of the final stress period
# get our contour levels using linspace
levels = list(range(1,36,3)) # we are setting 35 contours across the compete range of heads in the hds array forthe final step

In [ ]:
# same as before
mg = gwf.modelgrid
fig = plt.figure(figsize=(7,3)) # we are creating a figure object here so that we can dictate size
ax = fig.add_subplot(1, 1, 1, aspect="equal") # making sure our proportions are kept intact
pxs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0})
fa = pxs.plot_array(c) 
colorbar = plt.colorbar(fa, aspect=30, shrink= 0.8) # create a colorbar
fc = pxs.contour_array(c,
                       levels=levels,
                       colors="w",
                       linewidths=1.0, 
                       linestyles="-")
plt.clabel(fc, fmt="%4.3f", fontsize=7)
ax.set_title('Concentration density flow')
ax.set_xlabel('Length')
ax.set_ylabel('Height')
plt.tight_layout()
# lets save it to our plots folder
figname = os.path.join(plots_f,'Henry_concentration_density_flow.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'Henry_concentration_density_flow.pdf')
fig.savefig(figname,dpi=300) 

# Flow vectors

In [ ]:
bud = gwf.output.budget()  # get the flow model budget
spdis = bud.get_data(text='DATA-SPDIS')[-1]  # get the final time step
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf) # get component discharges, we need these for vectors

mg = gwf.modelgrid
fig = plt.figure(figsize=(7,3)) # we are creating a figure object here so that we can dictate size
ax = fig.add_subplot(1, 1, 1, aspect="equal") # making sure our proportions are kept intact
pxs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0})
fa = pxs.plot_array(c) 
colorbar = plt.colorbar(fa, aspect=30, shrink= 0.8) # create a colorbar
pxs.plot_vector(qx, qy, qz, color="white", kstep = 2, hstep = 2) # use kstep and hstep to change the number of vectors being plotted
plt.clabel(fc, fmt="%4.3f", fontsize=7)
ax.set_title('Concentration density flow vectors')
ax.set_xlabel('Length')
ax.set_ylabel('Height')
plt.tight_layout()
# lets save it to our plots folder
figname = os.path.join(plots_f,'Henry_concentration_density_vectors.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'Henry_concentration_density_vectors.pdf')
fig.savefig(figname,dpi=300) 

In [ ]:
# plotting a time series of figures
# first make our plot into a function
def my_plot(num,timestep):
    c = conc.get_data(timestep)
    fig = plt.figure(figsize=(7,3)) # we are creating a figure object here so that we can dictate size
    ax = fig.add_subplot(1, 1, 1, aspect="equal") # making sure our proportions are kept intact
    pxs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0})
    fa = pxs.plot_array(c) 
    colorbar = plt.colorbar(fa, aspect=30, shrink= 0.8) # create a colorbar
    fc = pxs.contour_array(c,
                           levels=levels,
                           colors="w",
                           linewidths=1.0, 
                           linestyles="-")
    plt.clabel(fc, fmt="%4.3f", fontsize=7)
    ax.set_title('Concentration density flow')
    ax.set_xlabel('Length')
    ax.set_ylabel('Height')
    plt.tight_layout()
    # lets save it to our plots folder
    figname = os.path.join(plots_f,'Henry_concentration_density_flow.png') # note use of the plots folder path.
    # If you want to change the file format then change the extension from .png to pdf or just do both
    fig.savefig(figname,dpi=300)
    figname = os.path.join(plots_f,f'conc_contours_{num:03d}.png') # note this is how we keep them sequential
    fig.savefig(figname,dpi=300)
    plt.close() # always a good idea when you create lots of plots

for i,output in enumerate(hds.get_kstpkper()[::10]): # every 10th output given our model output timing
    my_plot(i,output)

# Will then use ffmpeg from command line to stitch frames into video
# still trying to figure out how to best use it from python

# Using stress package concentrations

In [ ]:
gwf.remove_package("evt")
gwf.remove_package("rch")

In [ ]:
evt = flopy.mf6.ModflowGwfevta(gwf,
                               pname = "evt", # need this later
                               auxiliary="concentration",
                               surface = 1.0,
                               rate = 0.1,
                               depth = 0.5)

rch = flopy.mf6.ModflowGwfrcha(gwf,
                               pname = "rch",
                               auxiliary="concentration",
                               recharge=0.14256)

In [ ]:
sim.write_simulation()
sim.run_simulation()

In [ ]:
# get the concentrations 
# load the concentration file
concfile = os.path.join(model_f,"trans.ucn")
conc = flopy.utils.binaryfile.HeadFile(concfile, text='conc') 
# Note we are still using a HeadsFile object but reading a concentration file, you must tell it to look for "conc" as the array of interest

last_step = conc.get_kstpkper()[-1] # Now change this to conc instead of hds

c = conc.get_data(last_step)
# get the concentration from the final step of the final stress period
# get our contour levels using linspace
levels = list(range(1,36,3)) # we are setting 35 contours across the compete range of heads in the hds array forthe final step
# same as before
mg = gwf.modelgrid
fig = plt.figure(figsize=(7,3)) # we are creating a figure object here so that we can dictate size
ax = fig.add_subplot(1, 1, 1, aspect="equal") # making sure our proportions are kept intact
pxs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0})
fa = pxs.plot_array(c) 
colorbar = plt.colorbar(fa, aspect=30, shrink= 0.8) # create a colorbar
fc = pxs.contour_array(c,
                       levels=levels,
                       colors="w",
                       linewidths=1.0, 
                       linestyles="-")
plt.clabel(fc, fmt="%4.3f", fontsize=7)
ax.set_title('Concentration density flow')
ax.set_xlabel('Length')
ax.set_ylabel('Height')
plt.tight_layout()
# lets save it to our plots folder
figname = os.path.join(plots_f,'Henry_concentration_density_ET_RCH.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'Henry_concentration_density_ET_RCH.pdf')
fig.savefig(figname,dpi=300) 

In [ ]:
spc_dict = {}
spc_dict[0]=35.0 # no conc in first stress period

spc1 = flopy.mf6.ModflowUtlspca(gwt,concentration=spc_dict,
                         pname='spca1',
                         filename=f"{model_name2}_1spc.spca")
spc_dict = {}
spc_dict[0]=0.0
spc2 = flopy.mf6.ModflowUtlspca(gwt,concentration=spc_dict,
                         pname='spca2',
                         filename=f"{model_name2}_2spc.spca")

ssm =  flopy.mf6.ModflowGwtssm(gwt, 
                               sources=sourcerecarray,
                              fileinput=(['rch',f"{model_name2}_1spc.spca"]))
ssm.fileinput.append_list_as_record(['evt',f"{model_name2}_2spc.spca", 'MIXED']) # mixed means that we will have evapoconcentration

In [ ]:
sim.write_simulation()
sim.run_simulation()

In [ ]:
# get the concentrations 
# load the concentration file
concfile = os.path.join(model_f,"trans.ucn")
conc = flopy.utils.binaryfile.HeadFile(concfile, text='conc') 
# Note we are still using a HeadsFile object but reading a concentration file, you must tell it to look for "conc" as the array of interest

last_step = conc.get_kstpkper()[-1] # Now change this to conc instead of hds

c = conc.get_data(last_step)
# get the concentration from the final step of the final stress period
# get our contour levels using linspace
levels = list(range(1,36,3)) # we are setting 35 contours across the compete range of heads in the hds array forthe final step

bud = gwf.output.budget()  # get the flow model budget
spdis = bud.get_data(text='DATA-SPDIS')[-1]  # get the final time step
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf) # get component discharges, we need these for vectors

mg = gwf.modelgrid
fig = plt.figure(figsize=(7,3)) # we are creating a figure object here so that we can dictate size
ax = fig.add_subplot(1, 1, 1, aspect="equal") # making sure our proportions are kept intact
pxs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0})
fa = pxs.plot_array(c) 
colorbar = plt.colorbar(fa, aspect=30, shrink= 0.8) # create a colorbar
pxs.plot_vector(qx, qy, qz, color="white", kstep = 2, hstep = 2) # use kstep and hstep to change the number of vectors being plotted
plt.clabel(fc, fmt="%4.3f", fontsize=7)
ax.set_title('Concentration density flow vectors')
ax.set_xlabel('Length')
ax.set_ylabel('Height')
plt.tight_layout()
# lets save it to our plots folder
figname = os.path.join(plots_f,'Henry_concentration_density_ET_RCH.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'Henry_concentration_density_ET_RCH.pdf')
fig.savefig(figname,dpi=300) 

# Constant concentration package

In [ ]:
# going to add in concentration for tailings using spit_nodes
cnc_data = {}
pdata = []
for lay in range(20,31): # note this specific scenario considers concentration in layer 1.
    for col in range(20,31):
        pdata.append(((lay,0,col),27.0,'tailings'))
cnc_data[0]=pdata # note this will apply until end of simulation
cnc = flopy.mf6.ModflowGwtcnc(gwt,
                              boundnames=True,
                              save_flows=True,
                              maxbound=len(pdata) ,
                              stress_period_data = cnc_data,
                              filename="{}_1.cnc".format(model_name2),
                              pname = 'cnc_1')

In [ ]:
sim.write_simulation()
sim.run_simulation()

In [ ]:
# get the concentrations 
# load the concentration file
concfile = os.path.join(model_f,"trans.ucn")
conc = flopy.utils.binaryfile.HeadFile(concfile, text='conc') 
# Note we are still using a HeadsFile object but reading a concentration file, you must tell it to look for "conc" as the array of interest

last_step = conc.get_kstpkper()[-1] # Now change this to conc instead of hds

c = conc.get_data(last_step)
# get the concentration from the final step of the final stress period
# get our contour levels using linspace
levels = list(range(1,36,3)) # we are setting 35 contours across the compete range of heads in the hds array forthe final step
# same as before
mg = gwf.modelgrid
fig = plt.figure(figsize=(7,3)) # we are creating a figure object here so that we can dictate size
ax = fig.add_subplot(1, 1, 1, aspect="equal") # making sure our proportions are kept intact
pxs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0})
fa = pxs.plot_array(c) 
colorbar = plt.colorbar(fa, aspect=30, shrink= 0.8) # create a colorbar
fc = pxs.contour_array(c,
                       levels=levels,
                       colors="w",
                       linewidths=1.0, 
                       linestyles="-")
plt.clabel(fc, fmt="%4.3f", fontsize=7)
ax.set_title('Concentration density flow')
ax.set_xlabel('Length')
ax.set_ylabel('Height')
plt.tight_layout()
# lets save it to our plots folder
figname = os.path.join(plots_f,'Henry_concentration_density_CNC.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'Henry_concentration_density_CNC.pdf')
fig.savefig(figname,dpi=300) 

# Mass Source Loading package

In [ ]:
# going to add in concentration for tailings using spit_nodes
src_data = {}
pdata = []
for lay in range(0,2): # note this specific scenario considers concentration in layer 1.
    for col in range(50,60):
        pdata.append(((lay,0,col),1.0,'waste'))
src_data[0]=pdata # note this will apply until end of simulation
src = flopy.mf6.ModflowGwtsrc(gwt,
                              boundnames=True,
                              save_flows=True,
                              maxbound=len(pdata) ,
                              stress_period_data = src_data,
                              filename="{}_1.src".format(model_name2),
                              pname = 'src_1')

In [ ]:
sim.write_simulation()
sim.run_simulation()

In [ ]:
# get the concentrations 
# load the concentration file
concfile = os.path.join(model_f,"trans.ucn")
conc = flopy.utils.binaryfile.HeadFile(concfile, text='conc') 
# Note we are still using a HeadsFile object but reading a concentration file, you must tell it to look for "conc" as the array of interest

last_step = conc.get_kstpkper()[-1] # Now change this to conc instead of hds

c = conc.get_data(last_step)
# get the concentration from the final step of the final stress period
# get our contour levels using linspace
levels = list(range(1,36,3)) # we are setting 35 contours across the compete range of heads in the hds array forthe final step
# same as before
mg = gwf.modelgrid
fig = plt.figure(figsize=(7,3)) # we are creating a figure object here so that we can dictate size
ax = fig.add_subplot(1, 1, 1, aspect="equal") # making sure our proportions are kept intact
pxs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0})
fa = pxs.plot_array(c) 
colorbar = plt.colorbar(fa, aspect=30, shrink= 0.8) # create a colorbar
fc = pxs.contour_array(c,
                       levels=levels,
                       colors="w",
                       linewidths=1.0, 
                       linestyles="-")
plt.clabel(fc, fmt="%4.3f", fontsize=7)
ax.set_title('Concentration density flow')
ax.set_xlabel('Length')
ax.set_ylabel('Height')
plt.tight_layout()
# lets save it to our plots folder
figname = os.path.join(plots_f,'The_works.png') # note use of the plots folder path.
# If you want to change the file format then change the extension from .png to pdf or just do both
fig.savefig(figname,dpi=300)
figname = os.path.join(plots_f,'The_works.pdf')
fig.savefig(figname,dpi=300) 

# Thats all folks
That covers most of the commonly used solute transport packages. Advanced stress packages have there own transport variants of their flow packages which must be used if simulating transport with an advanced stress package. Hope you enjoyed this session and we'll see you for the next session which will cover the use of ZoneBudget and particle tracking with a few simple examples.
